doplnit graf zobrazujici data porizeni snimku

In [ ]:
# import libraries
from sentinelsat import SentinelAPI
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

from os import listdir
from os.path import join

In [ ]:
ROOT_DIR = '/media/sf_JD/DP'

In [ ]:
# read credentials from file
def read_credentials(filename):
    with open(filename, 'r') as txtfile:
        username, password = txtfile.read().split()
        return username, password

username, password = read_credentials('credentials_api_hub.txt')
print(username)
print(password)

api = SentinelAPI(username, password, 'https://apihub.copernicus.eu/apihub')

In [ ]:
# Change ROI to only include correct files for RO 73

In [ ]:
roi_path = join(ROOT_DIR, f'reference/2021/vyjezdy_2021_4326_singlepart.gpkg')
roi_gdf = gpd.read_file(roi_path)
x = roi_gdf.total_bounds
roi = f'POLYGON(({x[0]} {x[3]}, {x[2]} {x[3]}, {x[2]} {x[1]}, {x[0]} {x[1]}, {x[0]} {x[3]}))'

In [ ]:
# parameters
#roi = 'POLYGON((15.425 50.74, 15.515 50.74, 15.515 50.635, 15.425 50.635, 15.425 50.74))'
seasons = (('20210401', '20210930'), ('20220401', '20220930'))
orbits = (22, 95, 73, 146)

In [ ]:
# download files
for season in seasons:
    for orbit in orbits:
        query_result = api.query(
            date=season,
            platformname='Sentinel-1',
            raw=None,
            area=roi,
            area_relation='Contains',
            order_by=None,
            limit=200,
            offset=0,
            relativeorbitnumber=orbit,
            producttype='SLC',
            sensoroperationalmode='IW')
        print(f'Query for RO {orbit} in {season[0][:4]} returned {len(query_result)} responses.')
        download_dir = join(ROOT_DIR, f's1_download/{season[0][:4]}/{orbit}')
        api.download_all(query_result, directory_path=download_dir, n_concurrent_dl=4)

In [ ]:
# check files
for season in seasons:
    for orbit in orbits:
        download_dir = join(ROOT_DIR, f's1_download/{season[0][:4]}/{orbit}')
        download_list = [join(download_dir, filename) for filename in listdir(download_dir)]
        broken_files = api.check_files(paths=download_list, delete=False).keys()
        print(f'Files from {season[0][:4]}, RO {orbit} have {len(broken_files)} broken files.')

In [ ]:
world = gpd.read_file(
    gpd.datasets.get_path('naturalearth_lowres')
)
czechia = world[world.name=='Czechia']

In [ ]:
offset = 0

In [ ]:
offset += 1
# visualise image boundaries
fig, (axs) = plt.subplots(ncols=2, nrows=2, squeeze=False, sharex='all', sharey='all')
axs = axs.flatten()
fig.suptitle('Sentinel-1 observation bounds')

for idx, orbit in enumerate(orbits):
    season = seasons[1]
    query_result = api.query(
        date=season,
        platformname='Sentinel-1',
        raw=None,
        area=roi,
        area_relation='Contains',
        order_by=None,
        limit=200,
        offset=0,
        relativeorbitnumber=orbit,
        producttype='SLC',
        sensoroperationalmode='IW')
    
    obs_list = [query_result[obs_key]['footprint'] for obs_key in query_result.keys()]
    obs_list = obs_list[::-1]
    #obs_list = obs_list[offset:2+offset]
    obs_gs = gpd.GeoSeries.from_wkt(obs_list)
    
    #axs[idx].axis('equal')
    axs[idx].set_xlim([13,18])
    axs[idx].set_ylim([49,52])
    axs[idx].set_title(f'RO: {orbit}, #obs: {len(obs_list)}')
    czechia.boundary.plot(ax=axs[idx], color='black')
    obs_gs.boundary.plot(ax=axs[idx])
    roi_gdf.representative_point().plot(ax=axs[idx], color='red')

In [ ]:
# geopandas dataframe for managing downloaded products - way easier than a db

In [ ]:
def add_subswath_from_ro(gdf):
    gdf['subswath'] = ''    
    gdf.loc[gdf['relativeorbitnumber'] ==  22, 'subswath'] = "IW2"
    gdf.loc[gdf['relativeorbitnumber'] ==  73, 'subswath'] = "IW1"
    gdf.loc[gdf['relativeorbitnumber'] ==  95, 'subswath'] = "IW1"
    gdf.loc[gdf['relativeorbitnumber'] == 146, 'subswath'] = "IW3"

    return gdf

def add_bursts(gdf):
    if gdf.iloc[0]['beginposition'].year == 2021:
        if gdf.iloc[0]['relativeorbitnumber'] == 22:
            gdf['burst_first'] = [6,1,6,1,6,1,6,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
            gdf['burst_last'] =  [7,2,7,2,7,2,7,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2]
        if gdf.iloc[0]['relativeorbitnumber'] == 73:
            gdf['burst_first'] = [1] * 30
            gdf['burst_last'] =  [2] * 30
        if gdf.iloc[0]['relativeorbitnumber'] == 95:
            gdf['burst_first'] = [1,1,5,1,5,1,5,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
            gdf['burst_last'] =  [2,2,6,2,6,2,6,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2]
        if gdf.iloc[0]['relativeorbitnumber'] == 146:
            gdf['burst_first'] = [2] * 30
            gdf['burst_last'] =  [3] * 30
    if gdf.iloc[0]['beginposition'].year == 2022:
        if gdf.iloc[0]['relativeorbitnumber'] == 22:
            gdf['burst_first'] = [1] * 15
            gdf['burst_last'] =  [2] * 15
        if gdf.iloc[0]['relativeorbitnumber'] == 73:
            gdf['burst_first'] = [1] * 15
            gdf['burst_last'] =  [2] * 15
        if gdf.iloc[0]['relativeorbitnumber'] == 95:
            gdf['burst_first'] = [1] * 15
            gdf['burst_last'] =  [2] * 15
        if gdf.iloc[0]['relativeorbitnumber'] == 146:
            gdf['burst_first'] = [2] * 15
            gdf['burst_last'] =  [3] * 15
    return gdf

def downloaded_files_to_gdf(year, orbits):
    gdf_list = []
    for orbit in orbits:
        filenames_set = set([f'{fname[:-4]}*' for fname in listdir(join(ROOT_DIR, f's1_download/{year}/{orbit}'))])
        query_result = api.query(filename=filenames_set)
        
        gdf = api.to_geodataframe(query_result)
        gdf_removed_params = gdf.drop(columns=['title', 'link', 'link_alternative', 'link_icon', 'summary', 'ondemand', 'ingestiondate',
            'missiondatatakeid', 'orbitnumber', 'lastorbitnumber', 'lastrelativeorbitnumber', 'slicenumber',
            'format', 'status', 'sensoroperationalmode', 'swathidentifier', 'producttype', 'platformname',
            'platformidentifier', 'instrumentname', 'instrumentshortname', 'productclass', 'polarisationmode',
            'uuid', 'filename'])
        gdf_added_params = add_bursts(add_subswath_from_ro(gdf_removed_params))
        gdf_list.append(gdf_added_params)
    
    gdf_concat = pd.concat(gdf_list)
    return gdf_concat


In [ ]:
year = 2021
orbits = (22,73,95,146)

gdf_downloaded = downloaded_files_to_gdf(year, orbits)
gdf_downloaded.to_file(join(ROOT_DIR, f'reference/{year}/slc_products_{year}.gpkg'))